**install deps**

In [24]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


**load env**

In [25]:
from dotenv import load_dotenv
import os
load_dotenv()
esv_api_key = os.getenv("ESV_API_KEY")

In [32]:
import requests
import json

def get_passage(book, chapter):
    url = "https://api.esv.org/v3/passage/text/"
    headers = { "Authorization": f"Token {esv_api_key}" }
    params = { "q": book + str(chapter) }

    response = requests.get(url, headers=headers, params=params)

    with open(f"raw_books/{book}-{chapter}.json", "w") as f:
        json.dump(response.json(), f)

In [33]:
import re

def parse_esv_output(output):

    verse_pattern = re.compile(r"\[(\d+)\] (.*?) (?=\[\d+\]|\(ESV\))")
    verses = verse_pattern.findall(output)

    return {int(chapter): verse for chapter, verse in verses}

In [34]:
books = [("john", 21)]

for book_name, chapters in books:
    for chapter in range(1, chapters + 1):
        get_passage(book_name, chapter)

In [35]:

import os

for book in os.listdir("raw_books"):
    with open(f"raw_books/{book}") as f:
        data = json.load(f)

    verses = data["passages"][0]

    parsed = parse_esv_output(verses)

    book_name = book[:-5]

    with open(f"books/{book_name}.json", "w") as f:
        json.dump(parsed, f)


**type**

```yml
john:
  1:
    - In the beginning was the Word, and the Word was with God, and the Word was God.
    - He was in the beginning with God.
    - All things were made through him, and without him was not any thing made that was made.
    - In him was life, and the life was the light of men.
    - The light shines in the darkness, and the darkness has not overcome it.
```